In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
from dataloaders import load_ksdd2_custom as load_dataset
from dataloaders import set_loader_params, DATALOADER_PARAMS
from models import create_models
from train_orig import train_loop, get_optimizer
from eval_orig import compute_metrics, roc_auc
from eval_unet import Metrics

train_folder="kolektorsdd2/train"
test_folder="annotations/images_and_masks"



set_loader_params(height=640, width=232)

print(DATALOADER_PARAMS)

train_pos, train_neg_iter, test = load_dataset(
    train_folder,test_folder
)

reshaped_test = test.map(lambda img, mask, _, lbl : (tf.image.grayscale_to_rgb(img), mask, _, lbl))


# Shuffle the test dataset
# shuffled_test = test.shuffle(buffer_size=100)

seg_model = tf.keras.models.load_model('models/seg_model.h5')
seg_model.trainable = False

clf_model = tf.keras.models.load_model('models/clf_model.h5')
clf_model.trainable = False

seg_model.summary()
clf_model.summary()

num_show=10

# take random positive 
for img, mask, _, lbl in reshaped_test:
    if num_show==0:
        break
    if int(np.squeeze(lbl.numpy())) == 1:
        num_show-=1
        Sf, Sh = seg_model(img, training=False)
        Cp = clf_model(tf.concat([Sf, Sh], axis=3), training=False)

        Sh = tf.sigmoid(Sh)
        Cp = tf.sigmoid(Cp)

        Cp = tf.where(Cp > 0.5, 1, 0)

        print('Label: ', lbl.numpy())


        print('Prediction: ', Cp.numpy())

        
        # show image
        plt.figure(figsize=(15, 15))
        plt.subplot(1, 3, 1)
        plt.imshow(img[0])
        plt.axis('off')
        plt.title('image')

        # show mask
        plt.subplot(1, 3, 2)
        plt.imshow(mask[0])
        plt.axis('off')
        plt.title('mask')

        # show prediction
        plt.subplot(1, 3, 3)
        plt.imshow(Sh[0])
        plt.axis('off')
        plt.title('prediction')
        plt.show()
        

    
